## 1. installing required python libraries

In [1]:
%%capture
%pip install jinja2

## 2. generating documents

In [45]:
# merging user data and template into html files

import os #mkdir, remove
import json
import random
from jinja2 import Template

################################################################# read

def get_delegation_data(file_str='../input/delegate.json'):
    """reads the json dump file and returns as json variable"""
    with open(file_str, 'r', encoding='utf-8') as f:
        return json.load(f)['data']['delegation_processes']


def drop_old_runs(delegation_data):
    max_delegation_round = max([e['round'] for e in delegation_data])
    for run in delegation_data:
        if run['round'] == max_delegation_round:
            yield run

################################################################# validate
            
def is_valid_user(user_json):
    """very-very forgiving data quality check"""
    try:
        if (
            user_json['user_detail']['legal_name'] != '' and
            user_json['user_detail']['personal_identity_num'] != '' and
            (
                user_json['user_addresses'][0]['postal_cd'] != '' or
                user_json['user_addresses'][0]['city_name'] != '' or
                user_json['user_addresses'][0]['street_name'] != '')
        ):
            return True
        else:
            return False
    except:
        return False

    
def is_valid_analog(analog_json):
    """very-very forgiving data quality check"""
    try:
        if(
             analog_json['full_name'] != '' and
             analog_json['identity_num'] != '' and
             analog_json['address'] != ''):
            return True
        else:
            return False
    except:
        return False
             

def user_address_builder(user_json):
    """building a string from things that might or might not be there"""
    address = ''
    try:
        address += f"{user_json['user_addresses'][0]['postal_cd']} "
    except:
        pass
    try:
        address += f"{user_json['user_addresses'][0]['city_name']}, "
    except:
        pass
    try:
        address += f"{user_json['user_addresses'][0]['street_name']} "
    except:
        pass
    try:
        address += f"{user_json['user_addresses'][0]['street_type']['comment']} "
    except:
        pass
    try:
        address += f"{user_json['user_addresses'][0]['street_num']}"
    except:
        pass
    return address
        
        
################################################################# build

def doc_builder(delegation_run_data):
    for run_town in delegation_data:
        
        fileload = {'filename':f"{run_town['town_id']}.html",
                    'name' : run_town['town']['town_name'], 
                    'delegees' : []}

        for voting_location in run_town['town']['voting_locations']:
            if voting_location['user_2_voting_locations']:
                for user_location in voting_location['user_2_voting_locations']:

                    if is_valid_user(user_location['user']):
                        fileload['delegees'].append(
                            {
                                'name':user_location['user']['user_detail']['legal_name'],
                                'id':user_location['user']['user_detail']['personal_identity_num'],
                                'address':user_address_builder(user_location['user']),
                                'email_address':user_location['user']['email_address'],
                                'phone_num':user_location['user']['phone_num'],
                                'zone':voting_location['location_number']
                            }
                        )

                    elif is_valid_analog(user_location['analog_user']):
                         fileload['delegees'].append(
                             {
                                'name':user_location['analog_user']['full_name'],
                                'id':user_location['analog_user']['identity_num'],
                                'address':user_location['analog_user']['address'],
                                'email_address':user_location['analog_user']['email_address'],
                                'phone_num':user_location['analog_user']['phone_num'],
                                'zone':voting_location['location_number']
                             }
                         )                       
                    else:
                        pass # this should be error handling
            else:
                pass # empty delegation letter
            
            yield fileload

################################################################# merge

def read_letter_template(file_str='../input/megbizolevel_jinja_template.html'):
    with open(file_str, 'r', encoding='utf-8') as f:
        template = Template(f.read())
    return template


def generate_documents(fileload, jinja_template):
    path = "../output/"    
    for town in fileload:
        if town['delegees']:
            document = jinja_template.render(town = town)
            with open(path+town['filename'], 'w', encoding='utf-8') as f:
                    f.write(document)
 

In [46]:
generate_documents(
    doc_builder(
        drop_old_runs(
            get_delegation_data)),
    read_letter_template()
) 

In [51]:
# 399 entries, but not all of them have delegees
%ls /home/jupyter/20k_docgen/output/ | wc -l

319


## 3. installing libreoffice writer for pdf conversion

In [53]:
%%capture
!apt-get -qq install -y libreoffice-writer

## 4. converting HTMLs to PDFs and zipping them for download

In [ ]:
%%bash
cd /home/jupyter/20k_docgen/output/
for f in *.html ; do lowriter --headless --convert-to pdf "$f" ; done

In [ ]:
%%bash
zip /home/jupyter/20k_docgen/pub/500_megbizo_html.zip /home/jupyter/20k_docgen/output/*.pdf